# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../Output data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,24.39,79,100,6.36,PN,1707178724
1,1,fale old settlement,-9.3852,-171.2468,28.85,79,92,11.97,TK,1707178754
2,2,port-aux-francais,-49.3500,70.2167,6.11,78,98,12.71,TF,1707178722
3,3,port mathurin,-19.6833,63.4167,25.69,68,72,7.05,MU,1707178741
4,4,ad dabbah,18.0500,30.9500,15.10,19,87,6.37,SD,1707178881


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_data_df = city_data_df[(city_data_df['Cloudiness'] > 0) & \
                              (city_data_df['Cloudiness'] <= 10) & 
                                   (city_data_df['Max Temp'] >= 22) & \
                                   (city_data_df['Max Temp'] <= 35) & \
                                   (city_data_df['Wind Speed'] <= 5)]

# Drop any rows with null values
narrow_data_df = narrow_data_df.dropna(how='any')

# Display sample data
narrow_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
71,71,tocopilla,-22.0920,-70.1979,23.85,71,1,0.54,CL,1707178897
107,107,hadibu,12.6500,54.0333,24.24,77,6,3.09,YE,1707178761
113,113,kerikeri,-35.2268,173.9474,23.82,54,7,3.23,NZ,1707178608
192,192,thames,-37.1333,175.5333,25.39,50,1,1.22,NZ,1707178928
214,214,tam ky,15.5667,108.4833,22.23,90,2,1.39,VN,1707178888
240,240,lazaro cardenas,17.9583,-102.2000,26.24,69,1,2.68,MX,1707178940
251,251,payyannur,12.1000,75.2000,22.88,74,5,1.64,IN,1707178943
274,274,whangarei,-35.7317,174.3239,27.09,42,4,1.68,NZ,1707178747
318,318,waitara,-38.9250,174.2500,24.72,69,2,3.03,NZ,1707178959
355,355,aguililla,18.7333,-102.7333,27.05,24,4,0.55,MX,1707178968


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_data_df[['City' , 'Country' , 'Lat' , 'Lng' ,'Humidity']]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

C:\Users\theoc\AppData\Local\Temp\ipykernel_21216\1980465643.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
71,tocopilla,CL,-22.0920,-70.1979,71,
107,hadibu,YE,12.6500,54.0333,77,
113,kerikeri,NZ,-35.2268,173.9474,54,
192,thames,NZ,-37.1333,175.5333,50,
214,tam ky,VN,15.5667,108.4833,90,
240,lazaro cardenas,MX,17.9583,-102.2000,69,
251,payyannur,IN,12.1000,75.2000,74,
274,whangarei,NZ,-35.7317,174.3239,42,
318,waitara,NZ,-38.9250,174.2500,69,
355,aguililla,MX,18.7333,-102.7333,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"


params = {
    "apiKey":geoapify_key,
    "categories":categories	
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tocopilla - nearest hotel: hotel Isidora
hadibu - nearest hotel: No hotel found
kerikeri - nearest hotel: Avalon Resort
thames - nearest hotel: Kopu Station Hotel
tam ky - nearest hotel: Mường Thanh Quảng Nam
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
payyannur - nearest hotel: Hotel passport
whangarei - nearest hotel: Grand Hotel
waitara - nearest hotel: No hotel found
aguililla - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب


,City,Country,Lat,Lng,Humidity,Hotel Name
71,tocopilla,CL,-22.0920,-70.1979,71,hotel Isidora
107,hadibu,YE,12.6500,54.0333,77,No hotel found
113,kerikeri,NZ,-35.2268,173.9474,54,Avalon Resort
192,thames,NZ,-37.1333,175.5333,50,Kopu Station Hotel
214,tam ky,VN,15.5667,108.4833,90,Mường Thanh Quảng Nam
240,lazaro cardenas,MX,17.9583,-102.2000,69,Hotel Sol del Pacífico
251,payyannur,IN,12.1000,75.2000,74,Hotel passport
274,whangarei,NZ,-35.7317,174.3239,42,Grand Hotel
318,waitara,NZ,-38.9250,174.2500,69,No hotel found
355,aguililla,MX,18.7333,-102.7333,24,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name" , "Country"]
)

# Display the map
city_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)